In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import seaborn as sns; 
from IPython.display import HTML

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import r2_score

In [ ]:
#config parser
import configparser

sys.path.insert(0, '../Common/')
from AISDataManager import AISDataManager
import Constants as c
import HMUtils as hMUtil
import TimeUtils as timeUtils

#MyConfig.INI stores all the run time constants
config = configparser.ConfigParser()
config.read('../MyConfig.INI')

from joblib import Parallel, delayed
import multiprocessing
aISDM = AISDataManager()
numCores = multiprocessing.cpu_count()

In [ ]:
lonMin = (float)(config['TP_FIRST_ORDER']['LON_MIN'])
lonMax = (float)(config['TP_FIRST_ORDER']['LON_MAX'])

latMin = (float)(config['TP_FIRST_ORDER']['LAT_MIN'])
latMax = (float)(config['TP_FIRST_ORDER']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

increStep = (float)(config['TP_FIRST_ORDER']['INCR_STEP'])
incrRes = (int)(config['TP_FIRST_ORDER']['INCR_RES'])

fileDir = [\
        config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr00LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr01LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr02LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr03LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr04LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr05LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr06LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr07LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr08LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr09LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr10LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr11LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr12LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr13LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr14LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr15LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr16LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr17LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr18LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr19LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr20LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr21LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr22LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr23LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr24LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr25LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr26LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr27LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr28LE/" \
        ,config['TP_FIRST_ORDER']['SOURCE_DIR']+"HalfHr29LE/" \
      ]

dirToStore = (config['TP_FIRST_ORDER']['DEST_DIR'])

In [ ]:
heatMapGrid = hMUtil.generate_grid(lonMin, lonMax, latMin, latMax, increStep, incrRes)

In [ ]:
boundaryArray = heatMapGrid[2]
horizontalAxis = heatMapGrid[0]
verticalAxis = heatMapGrid[1]
totalStates = horizontalAxis.shape[0] * verticalAxis.shape[0]

In [ ]:
#1 to load existing data
loadExisting = 0
neighTPCount = []
for i in range(totalStates):
    fileToRead = dirToStore + str(i) + '.npy'
    if(loadExisting == 1):
        neighTPCount.append(np.load(fileToRead))
    else:
        neighTPCount.append(np.zeros((totalStates)))

In [ ]:
def get_index_from_lon_lat(lon,lat):
    retVal = -1
    for boundary in boundaryArray: 
        if(lon >= boundary[0]) and (lon < boundary[1]) \
            and (lat >= boundary[2]) and (lat < boundary[3]):
            retVal = boundary[4]
            break 
    return retVal

In [ ]:
def clear_tm_matrix():
    for i in range(totalStates):
        neighTPCount[i].fill(0)
        
def compute_transition_prob(firstNum,index = 0):
    fileName1 = fileDir[index] + str(firstNum) + '.csv'
    fileName2 = fileDir[index] + str(firstNum+1) + '.csv'

    #transition probability for first hour
    firstDF,_ = aISDM.load_data_from_csv(fileName1)
    #transition probability for next hour
    secondDF,_ = aISDM.load_data_from_csv(fileName2)
    
    #vessels in current hour
    currVessels = aISDM.get_list_of_unique_mmsi(firstDF)
    #vessels in next hour
    nextVessels = aISDM.get_list_of_unique_mmsi(secondDF)
    
    firstDF = firstDF.set_index('MMSI')
    secondDF = secondDF.set_index('MMSI')
    #compute vessels of interest
    #this are vessels which are of interest
    vesselsOfInterest = list(set(currVessels) & set(nextVessels))
    #iterate through every vessel
    #get the first LON and LAT coordinate for the first vessel
    for vessel in vesselsOfInterest:
        #get the LON and LAT value of the vessel
        #get the location where vessel is present
        #now we have the location
        vesselCurrIndex = get_index_from_lon_lat(firstDF.loc[vessel,'LON'],firstDF.loc[vessel,'LAT'])
        #now lets findout where this vessel is
        vesselNextIndex = get_index_from_lon_lat(secondDF.loc[vessel,'LON'],secondDF.loc[vessel,'LAT'])
        neighTPCount[vesselNextIndex][vesselCurrIndex] = neighTPCount[vesselNextIndex][vesselCurrIndex] + 1

In [ ]:
if(loadExisting == 0):
    clear_tm_matrix()
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,0)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,1)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,2)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,3)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,4)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,5)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,6)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,7)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,8)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break

    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,9)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,10)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,11)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,12)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,13)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,14)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,15)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,16)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,17)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,18)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,19)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,20)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,21)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,22)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,23)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,24)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,25)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,26)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,27)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,28)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break
            
    for fileCounter in range(17517):
        try:
            compute_transition_prob(fileCounter,29)
            print("Done Computing %d"%(fileCounter))
        except KeyboardInterrupt:
            break

In [ ]:
if(loadExisting == 0):
    #destination directory
    for neighb in range(totalStates):
        opFile = dirToStore + str(neighb)+'.npy'
        np.save(opFile, neighTPCount[neighb])

In [ ]:
#first lets make sum of probabilities
#this sum should be equal to tPCurrentSum
sumCount = np.zeros((totalStates))
for i in range(totalStates):
    sumCount = sumCount + neighTPCount[i]

In [ ]:
if(loadExisting == 0):
    #make transition probability matrix
    #A_{ij} i.e. trantition from i to j
    tpMatrix = np.zeros((sumCount.shape[0],sumCount.shape[0]))
    
    #we will first fill columns
    #then rows
    #just like numpy
    #so i will be row count
    #j will be column count
    # for i in range(1):
    for i in range(tpMatrix.shape[0]):
        for j in range(tpMatrix.shape[1]):
            if(sumCount[i] > 0):
                tpMatrix[i,j] = neighTPCount[j][i]/sumCount[i]
    #             print("YAY")
                if(tpMatrix[i,j] > 0):
                    print(tpMatrix[i,j])

In [ ]:
opFile = dirToStore + 'TPMFirst.npy'
if(loadExisting == 0):
    np.save(opFile,tpMatrix)
else:
    tpMatrix = np.load(opFile)

In [ ]:
#lets get the sparse reprentation of it
from scipy import sparse
tpMatrix = sparse.csr_matrix(tpMatrix)

In [ ]:
useAspect = 0
aspectRatio = 16.0/9.0
figWidth = 10
figHeight = 3

if useAspect == 1:
    figHeight = figWidth/aspectRatio

fig = plt.figure(figsize=(figWidth,figHeight))
ax1 = fig.add_axes([0.1, 0.15, 0.73, 0.73])

ax1.set_xlabel('States 0 to %d'%(horizontalAxis.shape[0] * verticalAxis.shape[0]))
ax1.set_ylabel('States 0 to %d'%(horizontalAxis.shape[0] * verticalAxis.shape[0]))

ax1.set_title('Transition Probability Matrix')

# iMShowHandler = ax1.imshow(tpMatrix,cmap=plt.get_cmap('YlGnBu'),vmax = 0.0001, extent=[0,399,0,399])
# iMShowHandler = ax1.imshow(tpMatrix,cmap=plt.get_cmap('YlGnBu'))
iMShowHandler = ax1.spy(tpMatrix)
# ax4 = fig.add_axes([0.80, 0.15, 0.01, 0.73])
# # fig.colorbar(iMShowHandler,ax4)
# fig.savefig("TrProbMatHalfHr.png")

In [ ]:
def get_vessel_init_status(lonStart,latStart):
    vesselState = np.zeros((horizontalAxis.shape[0]*verticalAxis.shape[0]))
    for boundary in boundaryArray :
        if((lonStart >= boundary[0]) \
            and (lonStart < boundary[1]) \
           
            and (latStart >= boundary[2]) \
            and (latStart < boundary[3]) \
          ):
            vesselState[boundary[4]] = 1.0
            break
    return vesselState

In [ ]:
#routine to store the matrixes
def store_higher_order_tp(noOfSteps):
    ipFile = dirToStore + 'TPM%d.npy'%((noOfSteps-1))
    tpMat = np.load(ipFile)
    tpMatNext = tpMat @ tpMatrix
    opFile = dirToStore + 'TPM%d.npy'%(noOfSteps)
    np.save(opFile,tpMatNext)
    print("Done storing the matrix")

In [ ]:
# if(loadExisting == 0):
# for i in range(3,20):
#     store_higher_order_tp(i)

In [ ]:
#routine to store the matrixes
'''
tpMat1 = tpMatrix.dot(tpMatrix)
print(type(tpMat1))
print("Done Computing 1")
opFile = dirToStore + 'TPM1.npz'
sparse.save_npz(opFile,tpMat1)
tpMat2 = tpMat1.dot(tpMatrix)
print("Done Computing 2")
opFile = dirToStore + 'TPM2.npz'
sparse.save_npz(opFile,tpMat2)

tpMat3 = tpMat2.dot(tpMatrix)
print("Done Computing 3")
opFile = dirToStore + 'TPM3.npz'
sparse.save_npz(opFile,tpMat3)

tpMat4 = tpMat3.dot(tpMatrix)
print("Done Computing 4")
opFile = dirToStore + 'TPM4.npz'
sparse.save_npz(opFile,tpMat4)

tpMat5 = tpMat4.dot(tpMatrix)
print("Done Computing 5")
opFile = dirToStore + 'TPM5.npz'
sparse.save_npz(opFile,tpMat5)

tpMat6 = tpMat5.dot(tpMatrix)
print("Done Computing 6")
opFile = dirToStore + 'TPM6.npz'
sparse.save_npz(opFile,tpMat6)

tpMat7 = tpMat6.dot(tpMatrix)
print("Done Computing 7")
opFile = dirToStore + 'TPM7.npz'
sparse.save_npz(opFile,tpMat7)

tpMat8 = tpMat7.dot(tpMatrix)
print("Done Computing 8")
opFile = dirToStore + 'TPM8.npz'
sparse.save_npz(opFile,tpMat8)

tpMat9 = tpMat8.dot(tpMatrix)
print("Done Computing 9")
opFile = dirToStore + 'TPM9.npz'
sparse.save_npz(opFile,tpMat9)

tpMat10 = tpMat9.dot(tpMatrix)
print("Done Computing 10")
opFile = dirToStore + 'TPM10.npz'
sparse.save_npz(opFile,tpMat10)

tpMat11 = tpMat10.dot(tpMatrix)
print("Done Computing 11")
opFile = dirToStore + 'TPM11.npz'
sparse.save_npz(opFile,tpMat11)

tpMat12 = tpMat11.dot(tpMatrix)
print("Done Computing 12")
opFile = dirToStore + 'TPM12.npz'
sparse.save_npz(opFile,tpMat12)

tpMat13 = tpMat12.dot(tpMatrix)
print("Done Computing 13")
opFile = dirToStore + 'TPM13.npz'
sparse.save_npz(opFile,tpMat13)

tpMat14 = tpMat13.dot(tpMatrix)
print("Done Computing 14")
opFile = dirToStore + 'TPM14.npz'
sparse.save_npz(opFile,tpMat14)

tpMat15 = tpMat14.dot(tpMatrix)
print("Done Computing 15")
opFile = dirToStore + 'TPM15.npz'
sparse.save_npz(opFile,tpMat15)

tpMat16 = tpMat15.dot(tpMatrix)
print("Done Computing 16")
opFile = dirToStore + 'TPM16.npz'
sparse.save_npz(opFile,tpMat16)

tpMat17 = tpMat16.dot(tpMatrix)
print("Done Computing 17")
opFile = dirToStore + 'TPM17.npz'
sparse.save_npz(opFile,tpMat17)

tpMat18 = tpMat17.dot(tpMatrix)
print("Done Computing 18")
opFile = dirToStore + 'TPM18.npz'
sparse.save_npz(opFile,tpMat18)

tpMat19 = tpMat18.dot(tpMatrix)
print("Done Computing 19")
opFile = dirToStore + 'TPM19.npz'
sparse.save_npz(opFile,tpMat19)

tpMat20 = tpMat19.dot(tpMatrix)
print("Done Computing 20")
opFile = dirToStore + 'TPM20.npz'
sparse.save_npz(opFile,tpMat20)

tpMat21 = tpMat20.dot(tpMatrix)
print("Done Computing 21")
opFile = dirToStore + 'TPM21.npz'
sparse.save_npz(opFile,tpMat21)

tpMat22 = tpMat21.dot(tpMatrix)
print("Done Computing 22")
opFile = dirToStore + 'TPM22.npz'
sparse.save_npz(opFile,tpMat22)
'''
print("Dont Re Compute it")

In [ ]:
#takes inital state vector
#and number of steps 
#uses transition probabilities to generate the path
def generate_path(vesselStart, noOfSteps):
    #uncomment this for smaller region
    '''
    tpMat = np.linalg.matrix_power(tpMatrix,noOfSteps)
    #convert vesselStart into 
    return (vesselStart.T @ tpMat).flatten()
    '''
    matFile = dirToStore + 'TPM%d.npy'%(noOfSteps)
    tpMat = np.load(matFile)
    return (vesselStart.T @ tpMat).flatten()

In [ ]:
# oneVessel = "../Data/M120_00_M190_50_34_12_34_24/MMSI/566952000_Sorted.csv"
oneVessel = "../Data/M121_00_M119_00_33_50_34_50/MMSI/566952000_Sorted.csv"
oneVesselData,_ = aISDM.load_data_from_csv(oneVessel)
print(oneVesselData.shape)
oneVesselLon = oneVesselData['LON']
oneVesselLat = oneVesselData['LAT']
plt.plot(oneVesselLon,oneVesselLat)

In [ ]:
oneVesselTime = oneVesselData['DateTime']
for i in range(oneVesselLon.shape[0]):
    print(oneVesselTime[i])

In [ ]:
# timeIntervalList = "../Data/M120_00_M190_50_34_12_34_24/TimeInterval/HalfHourIntvl1701To1702.txt"
timeIntervalList = "../Data/M121_00_M119_00_33_50_34_50/TimeInterval/HalfHourIntvl1701To1702.txt"
timeWindows = [line.rstrip('\n') for line in open(timeIntervalList)]

In [ ]:
#get the last instance of the position data 
#for one of the vessel
#and that can be used for the prediction
oneVesselLastData = pd.DataFrame()
#based on time stamps 
for timeSlot in timeWindows:
    temp = timeSlot.split(',')
    startTime = temp[0]
    endTime = temp[1]

    timelyDF = aISDM.filter_based_on_time_stamp(oneVesselData,'DateTime',startTime,endTime)
    oneVesselRows = timelyDF.shape[0]
    
    invertedTimelyDF = aISDM.inver_df(timelyDF)
    invertedTimelyDF = invertedTimelyDF.drop_duplicates(subset="MMSI")
    
    oneVesselLastData = oneVesselLastData.append(invertedTimelyDF, ignore_index = True)

In [ ]:
print(oneVesselLastData)

In [ ]:
#divide the journey into two parts
#part 1 is from 0 to 4 included
#part 2 is from 5 to 13 included
oneVesselLastDataOne = oneVesselLastData.iloc[0:21,:]
oneVesselLastDataTwo = oneVesselLastData.iloc[21:,:]

In [ ]:
print(oneVesselLastDataOne)

In [ ]:
print(oneVesselLastDataTwo)

In [ ]:
oneVesselLastDataOneLon = oneVesselLastDataOne["LON"]
oneVesselLastDataOneLat = oneVesselLastDataOne["LAT"]
plt.plot(oneVesselLastDataOneLon,oneVesselLastDataOneLat)

In [ ]:
oneVesselLastDataTwoLon = oneVesselLastDataTwo["LON"]
oneVesselLastDataTwoLat = oneVesselLastDataTwo["LAT"]
plt.plot(oneVesselLastDataTwoLon,oneVesselLastDataTwoLat)

In [ ]:
vesselState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[0],oneVesselLastDataOneLat.iloc[0])
useAspect = 0
aspectRatio = 16.0/9.0
figWidth = 10
figHeight = 8

fig = plt.figure(figsize=(figWidth,figHeight))
ax1 = fig.add_axes([0.1, 0.15, 0.49, 0.30])
ax2 = fig.add_axes([0.1, 0.55, 0.49, 0.30])
ax4 = fig.add_axes([0.8, 0.55, 0.01, 0.30])

if useAspect == 1:
    figHeight = figWidth/aspectRatio

def animate(i):
    vesselCurrPos = get_vessel_init_status(oneVesselLastDataOneLon.iloc[i],oneVesselLastDataOneLat.iloc[i])
    vesselCurrPos = np.reshape(vesselCurrPos,(verticalAxis.shape[0],horizontalAxis.shape[0]))
    
    vesselCurrPos = np.flip(vesselCurrPos,axis=0)

    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Lattitude')
    titleStr = 'Vessel Position %d'%((i))
    ax1.set_title(titleStr)
    ax1.set_xticks([lonMin,lonMax])
    ax1.set_yticks([latMin,latMax])

    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Lattitude')
    ax2.set_title('Transition Probability Projection')
    ax2.set_xticks([lonMin,lonMax])
    ax2.set_yticks([latMin,latMax])

    ax1.imshow(vesselCurrPos,cmap=plt.get_cmap('hot'),extent=[lonMin,lonMax,latMin,latMax])
    #sequence of vessel states
    vesselProbState = generate_path(vesselState,i)
    vesselProbState = np.reshape(vesselProbState,(verticalAxis.shape[0],horizontalAxis.shape[0]))
    vesselProbState = np.flip(vesselProbState,axis=0)
    
    iMShowHandler = ax2.imshow(vesselProbState,cmap=plt.get_cmap('YlGnBu'),extent=[lonMin,lonMax,latMin,latMax], vmax = 0.10)
    
    fig.colorbar(iMShowHandler,ax4)
    
# anim = animation.FuncAnimation(fig, animate,frames=20, interval=1000)
anim = animation.FuncAnimation(fig, animate,frames=20, interval=1000)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=1, metadata=dict(artist='Me'), bitrate=1800)

In [ ]:
anim.save('TPSeqOne.mp4', writer=writer)

In [ ]:
vesselState = get_vessel_init_status(oneVesselLastDataTwoLon.iloc[0],oneVesselLastDataTwoLat.iloc[0])
useAspect = 0
aspectRatio = 16.0/9.0
figWidth = 10
figHeight = 8

fig = plt.figure(figsize=(figWidth,figHeight))
ax1 = fig.add_axes([0.1, 0.15, 0.49, 0.30])
ax2 = fig.add_axes([0.1, 0.55, 0.49, 0.30])
ax4 = fig.add_axes([0.8, 0.55, 0.01, 0.30])

if useAspect == 1:
    figHeight = figWidth/aspectRatio

def animate(i):
    vesselCurrPos = get_vessel_init_status(oneVesselLastDataTwoLon.iloc[i],oneVesselLastDataTwoLat.iloc[i])
    vesselCurrPos = np.reshape(vesselCurrPos,(verticalAxis.shape[0],horizontalAxis.shape[0]))
    
    vesselCurrPos = np.flip(vesselCurrPos,axis=0)

    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Lattitude')
    titleStr = 'Vessel Position %d'%((i))
    ax1.set_title(titleStr)
    ax1.set_xticks([lonMin,lonMax])
    ax1.set_yticks([latMin,latMax])

    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Lattitude')
    ax2.set_title('Transition Probability Projection')
    ax2.set_xticks([lonMin,lonMax])
    ax2.set_yticks([latMin,latMax])

    ax1.imshow(vesselCurrPos,cmap=plt.get_cmap('hot'),extent=[lonMin,lonMax,latMin,latMax])
    #sequence of vessel states
    vesselProbState = generate_path(vesselState,i)
    vesselProbState = np.reshape(vesselProbState,(verticalAxis.shape[0],horizontalAxis.shape[0]))
    vesselProbState = np.flip(vesselProbState,axis=0)
    
    iMShowHandler = ax2.imshow(vesselProbState,cmap=plt.get_cmap('YlGnBu'),extent=[lonMin,lonMax,latMin,latMax], vmax = 0.10)
    
    fig.colorbar(iMShowHandler,ax4)
    
anim = animation.FuncAnimation(fig, animate,frames=20, interval=1000)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
anim.save('TPSeqTwo.mp4', writer=writer)

In [ ]:
oneFirstState = get_index_from_lon_lat(oneVesselLastDataOneLon.iloc[0],oneVesselLastDataOneLat.iloc[0])
print(oneFirstState)
oneFirstStateProb = tpMatrix[oneFirstState,:]
plt.stem(oneFirstStateProb, bottom = -0.01, label = "Transition Probabilities")
plt.legend()
plt.title("Stem Plot For State %d"%(oneFirstState))
plt.savefig("StemPlotOne.png")

In [ ]:
twoFirstState = get_index_from_lon_lat(oneVesselLastDataTwoLon.iloc[0],oneVesselLastDataTwoLat.iloc[0])
print(twoFirstState)
twoFirstStateProb = tpMatrix[twoFirstState,:]
plt.stem(twoFirstStateProb, bottom = -0.01, label = "Transition Probabilities")
plt.legend()
plt.title("Stem Plot For State %d"%(twoFirstState))
plt.savefig("StemPlotTwo.png")

In [ ]:
vesselState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[0],oneVesselLastDataOneLat.iloc[0])

useAspect = 0
aspectRatio = 16.0/9.0
figWidth = 10
figHeight = 4

fig = plt.figure(figsize=(figWidth,figHeight))
ax1 = fig.add_axes([0.1, 0.15, 0.83, 0.73])
ax4 = fig.add_axes([0.95, 0.25, 0.01, 0.50])

if useAspect == 1:
    figHeight = figWidth/aspectRatio
    
vesselProbState = vesselState.copy()
for i in range(1,20):
    vesselProbState = vesselProbState + generate_path(vesselState,i)

vesselProbState = np.reshape(vesselProbState,(verticalAxis.shape[0],horizontalAxis.shape[0]))
vesselProbState = np.flip(vesselProbState,axis=0)

ax1.set_xlabel('Longitude')
ax1.set_ylabel('Lattitude')
titleStr = 'Vessel Trajectory Density 1'
ax1.set_title(titleStr)
ax1.set_xticks([lonMin,lonMax])
ax1.set_yticks([latMin,latMax])
    
iMShowHandler = ax1.imshow(vesselProbState,cmap=plt.get_cmap('YlGnBu'),extent=[lonMin,lonMax,latMin,latMax], vmax = 0.10)
fig.colorbar(iMShowHandler,ax4)
fig.savefig('VesselTrajDen1.png')

In [ ]:
vesselState = get_vessel_init_status(oneVesselLastDataTwoLon.iloc[0],oneVesselLastDataTwoLat.iloc[0])

useAspect = 0
aspectRatio = 16.0/9.0
figWidth = 10
figHeight = 4

fig = plt.figure(figsize=(figWidth,figHeight))
ax1 = fig.add_axes([0.1, 0.15, 0.83, 0.73])
ax4 = fig.add_axes([0.95, 0.25, 0.01, 0.50])

if useAspect == 1:
    figHeight = figWidth/aspectRatio
    
vesselProbState = vesselState.copy()
for i in range(1,20):
    vesselProbState = vesselProbState + generate_path(vesselState,i)

vesselProbState = np.reshape(vesselProbState,(verticalAxis.shape[0],horizontalAxis.shape[0]))
vesselProbState = np.flip(vesselProbState,axis=0)

ax1.set_xlabel('Longitude')
ax1.set_ylabel('Lattitude')
titleStr = 'Vessel Trajectory Density 2'
ax1.set_title(titleStr)
ax1.set_xticks([lonMin,lonMax])
ax1.set_yticks([latMin,latMax])
    
iMShowHandler = ax1.imshow(vesselProbState,cmap=plt.get_cmap('YlGnBu'),extent=[lonMin,lonMax,latMin,latMax], vmax = 0.10)
fig.colorbar(iMShowHandler,ax4)
fig.savefig('VesselTrajDen2.png')

In [ ]:
def compute_heat_map(dF):
#     npHeatMap = np.ones((horizontalAxis.shape[0]*verticalAxis.shape[0]))
    npHeatMap = np.zeros((horizontalAxis.shape[0]*verticalAxis.shape[0]))
    for i in range(len(boundaryArray)):
        boundedDF = aISDM.filter_based_on_lon_lat(dF,boundaryArray[i][0]\
            ,boundaryArray[i][1]\
            ,boundaryArray[i][2]\
            ,boundaryArray[i][3]\
            )
        
        if(boundedDF.shape[0] > 0):
#             npHeatMap[i] = 0
            npHeatMap[i] = 1
    
    return npHeatMap

In [ ]:
print(oneVesselData.shape)

In [ ]:
lineCounter = 0
for i in range(oneVesselData.shape[0]):
    print(oneVesselData.iloc[i,16])
    lineCounter = lineCounter + 1
    print(lineCounter)

In [ ]:
oneVesselDataOne = oneVesselData.iloc[0:107,:].copy()
oneVesselDataTwo = oneVesselData.iloc[107:331,:].copy()

In [ ]:
for i in range(oneVesselDataOne.shape[0]):
    print(oneVesselDataOne.iloc[i,16])

In [ ]:
for i in range(oneVesselDataTwo.shape[0]):
    print(oneVesselDataTwo.iloc[i,16])

In [ ]:
vesselState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[0],oneVesselLastDataOneLat.iloc[0])
vesselActualTrajOne = compute_heat_map(oneVesselDataOne)
vesselActualTrajOne = np.reshape(vesselActualTrajOne,(verticalAxis.shape[0],horizontalAxis.shape[0]))
vesselActualTrajOne = np.flip(vesselActualTrajOne,axis=0)

useAspect = 0
aspectRatio = 16.0/9.0
figWidth = 10
figHeight = 4

fig = plt.figure(figsize=(figWidth,figHeight))
ax1 = fig.add_axes([0.1, 0.15, 0.83, 0.73])
ax4 = fig.add_axes([0.95, 0.25, 0.01, 0.50])

if useAspect == 1:
    figHeight = figWidth/aspectRatio
    
vesselProbState = vesselState.copy()
for i in range(1,5):
    vesselProbState = vesselProbState + generate_path(vesselState,i)

vesselProbState = np.reshape(vesselProbState,(verticalAxis.shape[0],horizontalAxis.shape[0]))
vesselProbState = np.flip(vesselProbState,axis=0)

ax1.set_xlabel('Longitude')
ax1.set_ylabel('Lattitude')
titleStr = 'Vessel Trajectory Density 1'
ax1.set_title(titleStr)
ax1.set_xticks([lonMin,lonMax])
ax1.set_yticks([latMin,latMax])
    
iMShowHandler = ax1.imshow(vesselProbState,cmap=plt.get_cmap('YlGnBu'),extent=[lonMin,lonMax,latMin,latMax], vmax = 0.10)
ax1.imshow(vesselActualTrajOne,cmap=plt.get_cmap('Reds'),extent=[lonMin,lonMax,latMin,latMax], vmax = 0.10, alpha = 0.2)
fig.colorbar(iMShowHandler,ax4)
fig.savefig('VesselTrajDen1.png')